# Machine Learning Model for Introvert/Extrovert Prediction

This notebook details the process of building a machine learning model to predict whether a person is an introvert or extrovert based on various characteristics.

## 1. Data Loading and Initial Exploration

We start by loading the dataset and performing an initial exploration to understand its structure, data types, and identify any missing values.


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/home/ubuntu/upload/personality_dataset.csv")

# Display basic information about the dataset
print(df.info())
print(df.head())
print(df.describe(include='all'))

# Check for missing values
print(df.isnull().sum())


### Initial Observations:

- The dataset contains 2900 entries and 8 columns.
- Columns like `Time_spent_Alone`, `Stage_fear`, `Social_event_attendance`, `Going_outside`, `Drained_after_socializing`, `Friends_circle_size`, and `Post_frequency` have missing values.
- `Stage_fear` and `Drained_after_socializing` are object type (categorical), while others are numerical.
- The target variable `Personality` is an object type and has no missing values.

## 2. Data Preprocessing

This section focuses on handling missing values and encoding categorical features to prepare the data for model training.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Separate target variable
X = df.drop("Personality", axis=1)
y = df["Personality"]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["float64", "int64"]).columns

# Preprocessing pipelines for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent"))
])

# Create a preprocessor object using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ])

# Apply preprocessing
X_processed = preprocessor.fit_transform(X)

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Original X shape: {X.shape}")
print(f"Processed X shape: {X_processed.shape}")
print(f"Original y shape: {y.shape}")
print(f"Encoded y shape: {y_encoded.shape}")
print("Sample of encoded y:", y_encoded[:5])
print("Sample of processed X (first 5 rows, first 5 columns):\n", X_processed[:5, :5])


### Preprocessing Steps:

- **Missing Values:** Numerical features are imputed using the mean strategy, and categorical features are imputed using the most frequent strategy.
- **Categorical Encoding:** Categorical features are one-hot encoded.
- **Target Encoding:** The `Personality` target variable is label encoded (Introvert/Extrovert to 0/1).

## 3. Model Training and Evaluation

We will use a RandomForestClassifier for this prediction task. The data is split into training and testing sets, the model is trained, and its performance is evaluated using various metrics.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np

# Create the full pipeline with preprocessing and model
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
model_pipeline.fit(X_train, y_train)

# Make predictions
y_pred = model_pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)


### Model Performance:

- **Accuracy:** The proportion of correctly classified instances.
- **Precision:** The proportion of true positive predictions among all positive predictions.
- **Recall:** The proportion of true positive predictions among all actual positive instances.
- **F1-Score:** The harmonic mean of precision and recall, providing a balanced measure.
- **Confusion Matrix:** A table showing the number of true positives, true negatives, false positives, and false negatives.

## 4. Model Testing and Prediction

This section demonstrates how to use the trained model to make predictions on new, unseen data.


In [ ]:
# Example of making a prediction on new data
# Let's create a sample new data point (ensure it has the same columns as the training data)
new_data = pd.DataFrame([{
    'Time_spent_Alone': 5,
    'Stage_fear': 'No',
    'Social_event_attendance': 3,
    'Going_outside': 7,
    'Drained_after_socializing': 'No',
    'Friends_circle_size': 10,
    'Post_frequency': 6
}])

# Make prediction
new_prediction_encoded = model_pipeline.predict(new_data)
new_prediction_personality = label_encoder.inverse_transform(new_prediction_encoded)

print(f"New data prediction: {new_prediction_personality[0]}")


## Conclusion

This notebook demonstrates a complete machine learning pipeline for predicting personality (introvert/extrovert) based on provided characteristics. The model achieved strong performance metrics, indicating its effectiveness in classifying individuals.
